# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import azureml
import joblib
import os

from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import DockerConfiguration
from azureml.core.dataset import Dataset


from azureml.core import Workspace, Experiment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.core.runconfig import EnvironmentDefinition, CondaDependencies

from azureml.train.hyperdrive import (
    BayesianParameterSampling,
    HyperDriveConfig,
    quniform,
    uniform,
    choice,
    PrimaryMetricGoal,
)

from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive-lgbm'

experiment=Experiment(ws, experiment_name)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

In [ ]:
# Create or attach a compute cluster

cluster_name = "canina-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('This compute target already exists.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2",
                                                          max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    print('A new compute target has been created.')
    
compute_target.wait_for_completion(show_output=True, min_node_count=0, timeout_in_minutes=10)

In [ ]:
train_dataset = Dataset.get_by_name(ws, name='train')
train_id = train_dataset.id

In [ ]:
auxi_dataset = Dataset.get_by_name(ws, name='auxi')
auxi_id = auxi_dataset.id

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# Configure Docker environment 
env = EnvironmentDefinition()
env.python.user_managed_dependencies = False
env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=["pandas", "numpy", "scipy", "scikit-learn", "lightgbm", "joblib"],
    python_version="3.6.2"
)
env.python.conda_dependencies.add_channel("conda-forge")

docker_config = DockerConfiguration(use_docker=True)

In [ ]:
# Training script
script_folder = "./hyperdrive"
train_script_name = "train.py"

src = ScriptRunConfig(source_directory=script_folder,
                      script=train_script_name,
                      arguments=["--train", train_id, "--auxi", auxi_id],
                      compute_target=compute_target,
                      environment=env,
                      docker_runtime_config=docker_config)

# Create the different params used during training
param_sampling = BayesianParameterSampling(
    {
        "--num-leaves": quniform(8, 128, 1),
        "--min-data-in-leaf": quniform(20, 500, 10),
        "--learning-rate": choice(
            1e-4, 1e-3, 5e-3, 1e-2, 1.5e-2, 2e-2, 3e-2, 5e-2, 1e-1
        ),
        "--feature-fraction": uniform(0.2, 1),
        "--bagging-fraction": uniform(0.1, 1),
        "--bagging-freq": quniform(1, 20, 1),
        "--max-rounds": quniform(50, 2000, 10),
        "--max-lag": quniform(3, 40, 1),
        "--window-size": quniform(3, 40, 1)
    }
)

hyperdrive_run_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=param_sampling,
    primary_metric_name="MAPE",
    primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
    max_total_runs=25,
    max_concurrent_runs=4
)

In [ ]:
# Submit the experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

In [ ]:
# We need to wait before retrieving the best model
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
# View the different models trained and their performance
hyperdrive_run.get_metrics()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run_hyperdrive = hyperdrive_run.get_best_run_by_primary_metric()

# Print the best model's Run Id and metrics
best_metrics = best_run_hyperdrive.get_metrics()
print('Run Id : ', best_run_hyperdrive.id)
print('Metrics :',best_metrics)

In [ ]:
# Get the best hyperparameter values
best_run_hyperdrive.get_details()['runDefinition']['arguments']

In [ ]:
# Display the saved file of the best model
best_run_hyperdrive.get_file_names()

In [ ]:
# Download the best model
id_file = 0
best_run_hyperdrive.download_file(best_run_hyperdrive.get_file_names()[id_file],
                                 output_file_path = './outputs/')

In [ ]:
joblib.load('./outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Register the best model
best_hyperdrive_model = best_run_hyperdrive.register_model(model_name = 'hyperdrive-lgb', model_path = 'outputs/model.joblib')

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

